In [1]:
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import joblib
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../Database/train_modified2.csv', index_col='ID')
X = train.drop(columns=['대출등급'])
y = train['대출등급']

In [3]:
X

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간,근로기간,주택소유상태,대출목적
ID,,,,,,,,,,,,,
TRAIN_00000,-0.563864,-0.220228,-0.039061,-0.852450,-0.376088,-0.800307,-0.972790,-0.038438,-0.072596,0,9,3,1
TRAIN_00001,-0.377979,0.370322,0.317312,-0.356112,-0.376088,-0.436819,-0.441089,-0.038438,-0.072596,1,2,1,10
TRAIN_00002,-0.610335,0.020813,-1.109217,-0.935172,-0.376088,0.103269,-0.627628,-0.038438,-0.072596,0,8,1,1
TRAIN_00003,-0.377979,0.382374,-0.434915,-0.852450,-0.376088,-0.483278,-0.624984,-0.038438,-0.072596,0,11,1,1
TRAIN_00004,-0.029446,-0.222879,0.635242,-0.521558,-0.376088,-0.577936,-0.634415,-0.038438,-0.072596,1,15,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_96289,-0.377979,1.165756,-1.033371,0.636562,-0.376088,0.147965,0.145240,-0.038438,-0.072596,0,2,1,3
TRAIN_96290,1.016156,0.382374,-1.466629,-0.025221,-0.376088,-0.232336,0.969657,-0.038438,-0.072596,1,2,1,10
TRAIN_96291,-0.377979,-0.099707,-0.834925,-0.273389,-0.376088,0.648623,-0.424788,-0.038438,-0.072596,0,0,1,3


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
cat_features=[i for i in range(9,13)]

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [5]:
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth = 10,
    l2_leaf_reg=3,
    task_type = 'CPU',
    loss_function = 'MultiClass',
    boosting_type = 'Ordered'
)
print(type(cat_model))
cat_model.fit(train_pool, eval_set=val_pool, verbose=5)

joblib.dump(cat_model, '../Files/cat_model.pkl')

<class 'catboost.core.CatBoostClassifier'>
0:	learn: 1.7714094	test: 1.7732518	best: 1.7732518 (0)	total: 7.71s	remaining: 2h 8m 23s
5:	learn: 1.3175666	test: 1.3189657	best: 1.3189657 (5)	total: 26.5s	remaining: 1h 13m 2s
10:	learn: 1.1202928	test: 1.1225476	best: 1.1225476 (10)	total: 41.6s	remaining: 1h 2m 19s
15:	learn: 1.0153005	test: 1.0181726	best: 1.0181726 (15)	total: 56.1s	remaining: 57m 29s
20:	learn: 0.9281632	test: 0.9315305	best: 0.9315305 (20)	total: 1m 16s	remaining: 59m 16s
25:	learn: 0.8659447	test: 0.8695244	best: 0.8695244 (25)	total: 1m 30s	remaining: 56m 12s
30:	learn: 0.8239941	test: 0.8273793	best: 0.8273793 (30)	total: 1m 42s	remaining: 53m 26s
35:	learn: 0.7844603	test: 0.7885585	best: 0.7885585 (35)	total: 1m 57s	remaining: 52m 32s
40:	learn: 0.7529451	test: 0.7583060	best: 0.7583060 (40)	total: 2m 11s	remaining: 51m 26s
45:	learn: 0.7247826	test: 0.7312525	best: 0.7312525 (45)	total: 2m 25s	remaining: 50m 27s
50:	learn: 0.7031607	test: 0.7101363	best: 0.7101

['../Files/cat_model.pkl']

In [6]:
print("CatBoost Accuracy:", accuracy_score(y_test, cat_model.predict(X_test)))

CatBoost Accuracy: 0.8636552440290758
